In [1]:
from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    pipeline,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
)
from datasets import load_dataset, Dataset
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd
import numpy as np
import re
import evaluate

In [2]:
data = load_dataset("zeroshot/twitter-financial-news-sentiment", split="train")

In [3]:
data[:3]

{'text': ['$BYND - JPMorgan reels in expectations on Beyond Meat https://t.co/bd0xbFGjkT',
  '$CCL $RCL - Nomura points to bookings weakness at Carnival and Royal Caribbean https://t.co/yGjpT2ReD3',
  '$CX - Cemex cut at Credit Suisse, J.P. Morgan on weak building outlook https://t.co/KN1g4AWFIb'],
 'label': [0, 0, 0]}

In [4]:
df = data.to_pandas()

In [5]:
def clean_tweet_text(text):
    # 1. Remove URLs (http:// or https://)
    text = re.sub(r"https?://\S+|www\.\S+", "", text)

    # 2. Remove Cashtags (e.g., $TSLA) and User Mentions (@user)
    # Removing these forces the model to learn the sentiment from the words,
    # not just the ticker/user which might be a good or bad stock.
    text = re.sub(r"[$@]\w+", "", text)

    # 3. Remove excessive whitespace
    text = re.sub(r"\s+", " ", text).strip()

    return text

In [6]:
df["cleaned_text"] = df["text"].apply(clean_tweet_text)

In [7]:
texts = df["cleaned_text"].tolist()
labels = df["label"].tolist()

label_map = {0: "negative", 1: "positive", 2: "neutral"}

labels_id = [label_map[i] for i in labels]

In [8]:
finbert = pipeline(
    "sentiment-analysis",
    model="ProsusAI/finbert",
    tokenizer="ProsusAI/finbert",
    device="mps",
    return_all_scores=True,
)

Device set to use mps
/Users/kheeern/AIAP/aiap/llm-huggingface/.venv/lib/python3.13/site-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
# datasetå= Dataset.from_pandas(df[["cleaned_text", "label"]])

In [ ]:
# dataset = dataset.train_test_split(test_size=0.2, seed=42)
# train_dataset = dataset["train"]
# test_dataset = dataset["test"]

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

In [ ]:
# def tokenize_function(examples):
#     return tokenizer(examples["cleaned_text"], padding="max_length", truncation=True)

In [ ]:
# tokenized_train = train_dataset.map(tokenize_function, batched=True)
# tokenized_eval = test_dataset.map(tokenize_function, batched=True)

In [ ]:
# tokenized_train.set_format(
#     type="torch", columns=["input_ids", "attention_mask", "label"]
# )
# tokenized_eval.set_format(
#     type="torch", columns=["input_ids", "attention_mask", "label"]
# )

In [ ]:
# num_labels = 3

# model = AutoModelForSequenceClassification.from_pretrained(
#     "ProsusAI/finbert", num_labels=num_labels
# )

In [ ]:
# accuracy_metric = evaluate.load("accuracy")

In [ ]:
# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     return accuracy_metric.compute(predictions=predictions, references=labels)

In [ ]:
# trainer_args = TrainingArguments(
#     output_dir="./finbert_twitter_results",
#     num_train_epochs=3,
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     warmup_steps=500,
#     weight_decay=0.01,
#     logging_dir="./logs",
#     logging_steps=100,
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     load_best_model_at_end=True,
# )

In [ ]:
# trainer = Trainer(
#     model=model,
#     args=trainer_args,
#     train_dataset=tokenized_train,
#     eval_dataset=tokenized_eval,
#     compute_metrics=compute_metrics,
#     processing_class=tokenizer
# )

In [ ]:
# trainer.train()

In [11]:
def get_sentiment_index(output_list: list) -> float:
    """
    Calculates the sentiment index from the model output.
    The pipeline returns a list of dictionaries for each class
    """
    sentiment_scores = {item["label"]: item["score"] for item in output_list}
    sentiment_index = sentiment_scores.get("positive", 0) - sentiment_scores.get(
        "negative", 0
    )
    return sentiment_index

In [ ]:
# Simulated Output for a single tweet ("Strong buy signal")
simulated_output = [
    {"label": "negative", "score": 0.1},
    {"label": "neutral", "score": 0.2},
    {"label": "positive", "score": 0.7},
]

In [13]:
sentiment_index = get_sentiment_index(simulated_output)
print(f"Calculated Sentiment Index: {sentiment_index}")

TypeError: list indices must be integers or slices, not str